#   Testat 1 (08.11.2018)

In [1]:
import numpy as np
from numpy import zeros, sqrt
import numpy.linalg as la
from rum.basics import matprint

In [2]:
A = np.array([[4.0, 2.0, -1.0, 0.0, 0.0, 0.0],
              [2.0, 4.0, 1.0, 1.0, 0.0, 1.0],
              [-1.0, 1.0, 5.0, 3.0, -1, 2.0],
              [0.0, 1.0, 3.0, 4.0, 1.1, 2.5],
              [0.0, 0.0, -1, 1.1, 2.4, 1.0],
              [0.0, 1.0, 2.0, 2.5, 1.0, 3.0]])
matprint(A)

 4 2 -1   0   0   0 
 2 4  1   1   0   1 
-1 1  5   3  -1   2 
 0 1  3   4 1.1 2.5 
 0 0 -1 1.1 2.4   1 
 0 1  2 2.5   1   3 


## (a) Zeigen Sie, dass die Matrix A positiv definit ist.

In [3]:
def test_spd(A):
    """ 
    Returns True if the A is symmetric positive-definite (spd)
    else returns False
    """
    # symmetry
    if not (A.transpose() == A).all():
        return False
    # positive eigenvalues
    lamdas, _ = la.eigh(A)
    for l in lamdas:
        if l <= 0:
            return False
    return True

In [4]:
if test_spd(A):
    print("A ist spd.")
else:
    print("A ist nicht spd.")

A ist spd.


## (b) Schreiben Sie ein Programm zur Lösung des linearen Gleichungssystems Ax = y. Verwenden Sie eine eigene Implementierung der Cholesky-Zerlegung.

In [5]:
def cholesky(A, verbose=False):
    """ 
    Cholesky decomposition of A
    Returns None if A is not spd. 
    Else returns lower diagonal matrix L.
    """
    if not test_spd(A):
        return None

    L = A.copy()
    N = A.shape[0]
    for i in range(N):
        for j in range(N):
            if j > i:
                L[i, j] = 0
            else:
                if verbose:
                    print("L{}{}=(".format(i, j), end="")
                sum_ = A[i, j]
                if verbose:
                    print("{}".format(A[i, j]), end="")
                for k in range(j):
                    if verbose:
                        print(" - {}*{}".format(L[i, k], L[j, k]), end="")
                    sum_ = sum_ - L[i, k]*L[j, k]
                if i == j:
                    if verbose:
                        print(")->sqrt")
                    L[i, j] = sqrt(sum_)
                else:
                    if verbose:
                        print(")/{}".format(L[j, j]))
                    L[i, j] = sum_ / L[j, j]
    return L

## Wenden Sie Ihre Routine an, um die Choleksy-Zerlegung von A zu berechnen, und lassen Sie die Matrix L ausgeben.

In [6]:
L = cholesky(A)
print("L=")
matprint(L, fmt='.2f')

L=
 2.00 0.00  0.00 0.00 0.00 0.00 
 1.00 1.73  0.00 0.00 0.00 0.00 
-0.50 0.87  2.00 0.00 0.00 0.00 
 0.00 0.58  1.25 1.45 0.00 0.00 
 0.00 0.00 -0.50 1.19 0.86 0.00 
 0.00 0.58  0.75 0.85 0.43 1.10 


## Überprüfen Sie die Zerlegung, indem Sie die Frobenius-Norm (norm(A,’fro’)) des Residuums R = A − LL T auswerten und auf der Konsole ausgeben lassen.

In [7]:
R = A - np.dot(L, L.T)
print("||R||2 = {:.3e}".format(la.norm(R, 'fro')))

||R||2 = 4.441e-16


## (c) Lösen Sie das Gleichungssystem Ax = LL T x = y durch Vorwärts-Rückwärtseinsetzen:

In [8]:
def forwardsubs(lower, rhs):
    "Solve y in Ly=b with lower triangular matrix L"
    N = len(rhs)
    y = zeros(N)
    for n in range(N):
        y[n] = (rhs[n] - sum([lower[n, i]*y[i] for i in range(n)]))/lower[n, n]
    return y


def backwardsubs(upper, rhs):
    "Solve x in Rx=y with upper triangular matrix R"
    N = len(rhs)
    x = zeros(N)
    for n in reversed(range(N)):
        x[n] = (rhs[n] - sum([upper[n, i]*x[i]
                              for i in reversed(range(n, N))]))/upper[n, n]
    return x

## (d) Testen Sie Ihr Programm mit der rechten Seite y = (1, −1, 5, 7, 6, −3) T . Berechnen Sie das Residuum r = Ax − y, sowie ||r||1 , ||r||2 und ||r||∞ .

In [9]:
b = np.array([1, -1, 5, 7, 6, -3])
L = cholesky(A)
y = forwardsubs(L, b)
x = backwardsubs(L.T, y)
with np.printoptions(precision=3):
    print("x={}".format(x))

x=[ 2.9   -1.329  7.942 -1.818  9.812 -7.607]


In [10]:
r = np.dot(A, x) - b
with np.printoptions(precision=3):
    print("r={}".format(r))

r=[ 8.882e-16  0.000e+00 -7.105e-15  0.000e+00  1.776e-15 -3.553e-15]


In [11]:
print("||r||1   = {:.3e}".format(la.norm(r, 1)))
print("||r||2   = {:.3e}".format(la.norm(r, 2)))
print("||r||inf = {:.3e}".format(la.norm(r, np.inf)))

||r||1   = 1.332e-14
||r||2   = 8.189e-15
||r||inf = 7.105e-15
